In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [2]:
BATCH, N, Interval = 10, 200, 10

In [3]:
def batch_generation(Batch, N, Interval, avg_hr):
    hr = np.full((Batch,N), avg_hr, dtype = np.float32)

    symptoms = np.random.randint(10, N-10, size=(Batch))

    for j in range(Batch):
        r = np.random.randint(0, 20)
        symptom = symptoms[j]
        for i in range(N):
            if(i > symptom-Interval and i < symptom+Interval):
                hr[j][i] += np.random.normal(30, 15) + r
            else:
                hr[j][i] += np.random.normal(5, 1) + r
    return hr, symptoms

In [4]:
hrs, labels = batch_generation(BATCH, N, Interval, 60)
hrs = torch.from_numpy(hrs)
labels = torch.from_numpy(labels)
hrs = torch.tensor(hrs, dtype=torch.float32, requires_grad=True)
labels = torch.tensor(labels, dtype=torch.float32, requires_grad=True) 




/var/folders/06/9qvqpg0x1wsdlqmwnym9zc880000gn/T/ipykernel_9567/510098676.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hrs = torch.tensor(hrs, dtype=torch.float32, requires_grad=True)
/var/folders/06/9qvqpg0x1wsdlqmwnym9zc880000gn/T/ipykernel_9567/510098676.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32, requires_grad=True)


In [5]:
print(hrs[0])
print(labels[0])

tensor([ 64.2651,  65.4136,  65.8331,  66.7290,  67.3272,  66.4067,  64.9957,
         67.4860,  64.5169,  67.0564,  65.7605,  66.4617,  66.4117,  66.0160,
         64.4295,  63.5900,  65.5018,  64.4915,  65.9889,  64.8535,  65.4155,
         64.2603,  65.8900,  64.9988,  66.6458,  66.8169,  66.5365,  66.1143,
         65.9945,  65.5478,  66.3190,  66.8661,  66.0154,  66.9622,  66.1272,
         67.3266,  66.4432,  66.6986,  65.9474,  67.0354,  67.2609,  65.4581,
         66.0792,  66.4870,  65.8144,  65.9746,  66.9499,  66.1863,  65.0372,
         67.0373,  67.0580,  64.7642,  66.9978,  65.6272,  66.5872,  68.3308,
         82.3761,  88.1481,  87.7614,  92.5481,  76.7669,  92.5072,  80.5111,
         91.4765,  93.7366,  93.4136, 127.5172,  89.8828,  93.3531,  97.8441,
         78.4274,  82.1892, 107.0730,  97.4311,  64.7544,  65.4815,  66.6969,
         66.5430,  65.9752,  66.3743,  65.0317,  66.7442,  65.9354,  67.0313,
         66.5240,  65.1840,  66.2398,  67.5570,  66.9082,  65.57

In [6]:
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

input_size = 1
sequence_length = 20
hidden_size = 32 
num_layers = 1
device = torch.device("cpu")


In [7]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
        

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float32)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float32)
        
        out, _ = self.lstm(x, (h0,c0))  
        #out, _ = self.rnn(x,h0)
        out = out[:, -1, :]
        out = self.linear(out)
        return out



In [8]:
model = LSTM(input_size, hidden_size, num_layers, num_classes)
model.to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)


In [9]:
steps = 30 
recovery_period = 40
one_label_period = 10
balance = 6
for i in range(30):
    losses = []
    for batch in range(BATCH):
        symptom = labels[batch].item()
        #hr size = 200
        
        for s in range(0, N-steps, 4):
            

            if(s+steps > symptom and s+steps < symptom + recovery_period):
                continue

            x = hrs[batch][s:s+steps]
            
            x = torch.unsqueeze(x, 0)
            x = torch.unsqueeze(x, -1)

            y_pred = model(x)
            y_pred = torch.squeeze(y_pred)

            y = torch.tensor(0, dtype=torch.float32)
            if(s + steps >= symptom - one_label_period):
                y = torch.tensor(1, dtype=torch.float32)

                loss =  balance*criterion(y_pred, y)
               
            else:
                 print("0 label")
                print(y_pred.item())
                loss = criterion(y_pred, y)
            

            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    print("avg_loss", torch.tensor(losses).mean().item())
    

1 label
0.5063743591308594
1 label
0.5078693628311157
1 label
0.5104171633720398
1 label
0.5118489861488342
1 label
0.5131303668022156
1 label
0.5146094560623169
1 label
0.515985906124115
1 label
0.5172942280769348
1 label
0.5186963081359863
1 label
0.5203441977500916
1 label
0.5214691758155823
1 label
0.5226646065711975
1 label
0.5242106914520264
1 label
0.5252860188484192
1 label
0.5267426371574402
1 label
0.5280176997184753
1 label
0.5292220115661621
1 label
0.5302015542984009
1 label
0.5316812992095947
1 label
0.5331193804740906
1 label
0.534309983253479
1 label
0.5355927348136902
1 label
0.5363945960998535
1 label
0.5377496480941772
1 label
0.539165735244751
1 label
0.5395799875259399
1 label
0.5385335087776184
1 label
0.5423526167869568
1 label
0.5390615463256836
1 label
0.5404890179634094
1 label
0.5422576665878296
1 label
0.5434211492538452
1 label
0.5445799231529236
1 label
0.5459526777267456
1 label
0.5473057627677917
1 label
0.5483331680297852
1 label
0.5496385097503662
1 la

KeyboardInterrupt: 